In [1]:
import numpy as np
import pandas as pd 
import glob
from tqdm import tqdm
import hashlib
import datetime

from rdflib import Namespace, Literal, URIRef
from rdflib.graph import Graph, ConjunctiveGraph
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

In [2]:
# today
today = datetime.date.today()
print(f"마지막으로 실행한 날짜는 {today} 입니다.")

마지막으로 실행한 날짜는 2022-09-06 입니다.


### (1) 건축물대장 표제부 데이터 불러오기

In [5]:
%%time

columns = ["관리건축물대장PK","대장구분코드","대장구분코드명","대장종류코드","대장종류코드명","대지위치", \
            "도로명대지위치","건물명","시군구코드","법정동코드","대지구분코드","번","지","특수지명", \
            "블록","로트","외필지수","새주소도로코드","새주소법정동코드","새주소지상지하코드","새주소본번", \
            "새주소부번","동명","주부속구분코드","주부속구분코드명","대지면적(㎡)","건축면적(㎡)", \
            "건폐율(%)","연면적","용적률산정연면적","용적률","구조코드","구조코드명","기타구조", \
            "주용도코드","주용도코드명","기타용도","지붕코드","지붕코드명","기타지붕","세대수(세대)", \
            "가구수(가구)","높이(m)","지상층수","지하층수","승용승강기수","비상용승강기수","부속건축물수", \
            "부속건축물면적","총동연면적","옥내기계식대수(대)","옥내기계식면적","옥외기계식대수(대)", \
            "옥외기계식면적","옥내자주식대수(대)","옥내자주식면적","옥외자주식대수(대)","옥외자주식면적", \
            "허가일","착공일","사용승인일","허가번호년","허가번호기관코드","허가번호기관코드명", \
            "허가번호구분코드","허가번호구분코드명","호수(호)","에너지효율등급","에너지절감율", \
            "에너지EPI점수","친환경건축물등급","친환경건축물인증점수","지능형건축물등급","지능형건축물인증점수", \
            "생성일자","내진설계적용여부","내진능력"]

df = pd.read_csv("data/rawdata/building/표제부/mart_djy_03.txt", sep = "\|", engine='python', dtype=str, \
                    on_bad_lines='skip', keep_default_na=False, encoding = "cp949", header=None, names=columns)
df = df.replace({np.nan: None})

# preprocessing
df[df.columns] = df.apply(lambda x: x.str.strip())

# 관리건출물대장PK가 없는 row는 삭제
index = df[df['관리건축물대장PK'] == None].index
df = df.drop(index)

print(f"총 {df.shape[0]} 개의 행수가 있습니다.")

총 7905108 개의 행수가 있습니다.
Wall time: 7min 22s


In [6]:
# 속성 rdf 파일을 만들기 위해 속성으로만 df 만들기

col = list(df.columns)
col_df = pd.DataFrame(col, columns=["속성명"])

### (1-2) 데이터 쪼개기

In [14]:
# 시도 코드 만들기

df['시도코드'] = df['시군구코드'].apply(lambda s : str(s)[:2])
df['시도코드'].unique()

array(['27', '28', '30', '31', '45', '47', '43', '44', '11', '41', '46',
       '50', '48', '26', '29', '42', '36'], dtype=object)

In [15]:
print(os.getcwd())

c:\Users\datahub\Documents\GitHub\seon-dask-bigdata-tutorial


In [27]:
region_code = list(df['시도코드'].unique())
final_num = 0

for code in region_code : 
    df_temp = df[df['시도코드'] == code]
    df_temp.drop('시도코드', axis = 1, inplace=True)
    df_temp.to_csv(f'data/split_data/{code}.csv', index = False)
    
    num_row = len(df_temp)
    
    print(code, num_row)
    
    final_num += num_row
    
print(final_num) # 전체 df 수량과 맞는지 확인

c:\Users\datahub\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


27 260713
28 230930
30 144160
31 152299
45 505344
47 911810
43 446735
44 592869
11 595368
41 1286913
46 727574
50 208477
48 794192
26 362199
29 188306
42 459410
36 37809
7905108


### (2) RDF 변환

In [5]:
# namespace 
buid_def = "http://data.kisti.re.kr/administration/building/def/"
buid_id = "http://data.kisti.re.kr/administration/building/id/"
BUID = Namespace(buid_def)

# function (convert cell to rdf)
def cell(store, s, p, df_col, datatype = None, lang = None):
    if df_col != None:
        store.add((s, p, Literal(df_col, datatype=datatype, lang = lang)))
        
def uri(store, s, p, df_col, objClass = None, objURI = None) :
    if df_col != None :
        obj = URIRef(objURI + df_col) 
        store.add((s, p, obj))
        if objClass != None :
            store.add((obj, RDF.type, objClass))

### (3) 속성 만들기 

In [4]:
%%time

# generate Graph()
g = Graph()
g.bind("buid", BUID)

#vocab 
for idx, row in tqdm(col_df.iterrows(), total=col_df.shape[0]):
    # base def 
    bui_uri = URIRef(buid_def + f"pp{idx+1}")
    g.add((bui_uri, RDFS.label, Literal(row["속성명"])))

# save as ttl
g.serialize(destination=f"final-data/ttl/building-property-pyojebu-{today}.ttl", format="turtle")

NameError: name 'BUID' is not defined

### (4) 건축물대장 표제부 변환하기

In [7]:
%%time

# generate Graph()
g = Graph()
g.bind("buid", BUID)

for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    # base id 
    uri_buid = URIRef(buid_id + row["관리건축물대장PK"])

    # type
    g.add((uri_buid, RDF.type, BUID.Building))

    # literal
    for p in range(len(df.columns)):
        cell(g, uri_buid, BUID[f"pp{p+1}"], row[f"{df.columns[p]}"])

# the number of triples
print(f"총 {len(g)} 개의 트리플이 있습니다.")

# save as ttl
g.serialize(destination=f"final-data/ttl/building-pyo-{today}.ttl", format="ttl")

 30%|██▉       | 2360972/7905108 [1:55:58<3:15:41, 472.18it/s]   

In [32]:
df_row = pd.DataFrame()
df_row = df_row.append(pd.Series(list_row), ignore_index = True)

In [6]:
%%time
# 시도별로 작업하는 수정 코드

#2022-09-02기준 
#'11','26','27','28','29','30','31','42','45','47','43','44','41','46',완료
region_code = ['50','48','36']

df_row = pd.DataFrame()

for code in region_code :
    print(code, "시작합니다." )
    df_region = pd.read_csv(f'data/split_data/{code}.csv', dtype = str)
    df_region.replace({np.NaN : None}, inplace= True)
    
    # generate Graph()
    g = Graph()
    g.bind("buid", BUID)

    for idx, row in tqdm(df_region.iterrows(), total=df_region.shape[0]):
        # base id 
        uri_buid = URIRef(buid_id + row["관리건축물대장PK"])

        # type
        g.add((uri_buid, RDF.type, BUID.Building))

        # literal
        for p in range(len(df_region.columns)):
            cell(g, uri_buid, BUID[f"pp{p+1}"], row[f"{df_region.columns[p]}"])
    
    # the number of triples
    list_row = [code, int(len(g))]
    df_row = df_row.append(pd.Series(list_row), ignore_index = True)
    print(f"{code} 지역에는 총 {list_row[1]} 개의 트리플이 있습니다.")

    # save as ttl
    g.serialize(destination=f"final-data/ttl/{code}-building-pyo-{today}.ttl", format="ttl")

50 시작합니다.


100%|██████████| 208477/208477 [05:56<00:00, 585.53it/s]


50 지역에는 총 13278902 개의 트리플이 있습니다.
48 시작합니다.


100%|██████████| 794192/794192 [23:32<00:00, 562.44it/s]   


48 지역에는 총 50504154 개의 트리플이 있습니다.
36 시작합니다.


100%|██████████| 37809/37809 [00:59<00:00, 635.64it/s]


36 지역에는 총 2446565 개의 트리플이 있습니다.
Wall time: 1h 22min 57s


In [5]:
df_row.to_excel('data/split_data/num_of_data.xlsx', index = False)

In [ ]:
region_code = ['27', '28', '30', '31', '45', '47', '43', '44', '11', '41', '46', '50', '48', '26', '29', '42', '36']
region_code_label = ['대구광역시', '인천광역시', '대전광역시', '울산광역시', '전라북도', '경상북도', '충청북도', '충청남도', '서울특별시', 
                     '경기도', '전라남도', '제주특별자치도', '경상남도', '부산광역시', '광주광역시', '강원도', '세종특별자치시']



In [7]:
import smtplib
from email.mime.text import MIMEText

smtp = smtplib.SMTP('smtp.gmail.com', 587)
smtp.ehlo()      # say Hello
smtp.starttls()  # TLS 사용시 필요
smtp.login('haram9553@gmail.com', '888888')

to_mail_list = ['chaeeunsong.hike@gmail.com', 'haram9553@gmail.com']

for to_mail in to_mail_list : 
    msg = MIMEText('변환 완료')
    msg['Subject'] = '표제부 데이터 변환이 끝났습니다.'
    msg['To'] = to_mail
    smtp.sendmail('haram9553@gmail.com', to_mail, msg.as_string())
 
smtp.quit()

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials 188-20020a6216c5000000b005368fcfb7f8sm413829pfw.89 - gsmtp')

# 표제부 raw 데이터 수량

In [3]:
#2022-09-02기준 
#'11','26','27','28','29','30','31','42','45','47','43','44','41','46',완료
region_code = ['11','26','27','28','29','30','31','42','45','47','43','44','41','46','50','48','36']

df_row = pd.DataFrame()

for code in region_code :
    df_region = pd.read_csv(f'data/split_data/{code}.csv', dtype = str)
    df_region.replace({np.NaN : None}, inplace= True)
    print(code, "행 수: ", len(df_region))

11 행 수:  595368
26 행 수:  362199
27 행 수:  260713
28 행 수:  230930
29 행 수:  188306
30 행 수:  144160
31 행 수:  152299
42 행 수:  459410
45 행 수:  505344
47 행 수:  911810
43 행 수:  446735
44 행 수:  592869
41 행 수:  1286913
46 행 수:  727574
50 행 수:  208477
48 행 수:  794192
36 행 수:  37809


In [4]:
#2022-09-02기준 
#'11','26','27','28','29','30','31','42','45','47','43','44','41','46',완료
region_code = ['11','26','27','28','29','30','31','42','45','47','43','44','41','46','50','48','36']

df_row = pd.DataFrame()

for code in region_code :
    df_region = pd.read_csv(f'data/split_data/{code}.csv', dtype = str)
    df_region.replace({np.NaN : None}, inplace= True)
    print(code, "행 수: ", df_region.shape)

11 행 수:  (595368, 77)
26 행 수:  (362199, 77)
27 행 수:  (260713, 77)
28 행 수:  (230930, 77)
29 행 수:  (188306, 77)
30 행 수:  (144160, 77)
31 행 수:  (152299, 77)
42 행 수:  (459410, 77)
45 행 수:  (505344, 77)
47 행 수:  (911810, 77)
43 행 수:  (446735, 77)
44 행 수:  (592869, 77)
41 행 수:  (1286913, 77)
46 행 수:  (727574, 77)
50 행 수:  (208477, 77)
48 행 수:  (794192, 77)
36 행 수:  (37809, 77)
